In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import scipy
import scipy.stats
import statsmodels
from statsmodels.stats.multitest import multipletests


def filter_functions(callstacks, names):
  result = [[function for function in callstack if function in names] for callstack in callstacks]
  return [callstack for callstack in result if len(callstack) > 1]

class Capture:
  def __init__(self, callstack_csv_path, frametrack_csv_path): 
    self.callstack_csv = pd.read_csv(callstack_csv_path).to_numpy()
    self.frametrack_csv = pd.read_csv(frametrack_csv_path).to_numpy()
    self.frame_durations = self.frametrack_csv[:, 4]
    
    # last_init_frame = np.max((self.frame_durations > 50*1e6).nonzero())
    # init_end_time = self.frametrack_csv[last_init_frame, 3]
    # self.frametrack_csv = self.frametrack_csv[self.frametrack_csv[:,2] > init_end_time,:]
    # self.callstack_csv = self.callstack_csv[self.callstack_csv[:,1] > init_end_time,:]
    # self.frame_durations = self.frametrack_csv[:, 4]
    
    self.callstack_csv = self.callstack_csv[self.callstack_csv[:, 1].argsort()] # sort by timestamp
    self.frametrack_csv = self.frametrack_csv[self.frametrack_csv[:, 3].argsort()] # sort by end
    self.frame_index_to_callstacks = self.get_frame_index_to_callstacks()

  def filter_functions(self, names):
    filterred = {}
    for frame_index, callstacks in self.frame_index_to_callstacks.items():
      filterred[frame_index] = filter_functions(callstacks, names)
    self.frame_index_to_callstacks = filterred

  # returns a dict frame_index -> List[Callstack]
  def get_frame_index_to_callstacks(self): 
    frame_index_to_callstacks = {}
    frame_index = 0
    frame_ends = self.frametrack_csv[:, 3]
    frame_end = frame_ends[frame_index]
    frame_index_to_callstacks[frame_index] = []
    callstack_names = [s.split('/') for s in  self.callstack_csv[:, 2]]
    timestamps = self.callstack_csv[:, 1]
    for i in range(self.callstack_csv.shape[0]):
      timestamp = timestamps[i]

      while timestamp > frame_end:
        if frame_index + 1 == self.frametrack_csv.shape[0]: 
          return frame_index_to_callstacks
        frame_index += 1
        frame_end = frame_ends[frame_index]
        frame_index_to_callstacks[frame_index] = [] 
      frame_index_to_callstacks[frame_index].append(callstack_names[i])
    return frame_index_to_callstacks

  def get_exclusive_count(self, function_name):
    exclusive_count = 0;
    for callstacks in self.frame_index_to_callstacks.values():
      for callstack in callstacks:
        if (len(callstack) > 0 and callstack[0] == function_name):
          exclusive_count += 1
    return exclusive_count

  def avg_frame_time(self):
    return self.frame_durations.mean() / 1e6

  def total_callstacks(self):
    return  sum(len(s) for s in self.frame_index_to_callstacks.values())

  def get_exclusive_rate(self):
    name_to_hits = {}
    for callstacks in self.frame_index_to_callstacks.values():
      for callstack in callstacks:
        head = callstack[0]
        name_to_hits.setdefault(head, 0)
        name_to_hits[head] += 1
    total_callstacks = self.total_callstacks()
    return defaultdict(lambda : 0, dict([(name, (hits / total_callstacks)) for name, hits in name_to_hits.items()]))

  def time_per_frame(self): 
    mean_frame_time = self.avg_frame_time()
    return defaultdict(lambda : 0, dict([(name, (mean_frame_time  * rate)) for name, rate in self.get_exclusive_rate().items()]))

  def function_names(self):
    result = []
    for callstacks in self.frame_index_to_callstacks.values():
      for callstack in callstacks:
        result += callstack
    return list(set(result))


In [2]:
capture1 = Capture("~/cumbia_callstack1_thread.csv", "~/cumbia_frametrack1.csv")
capture2 = Capture("~/cumbia_callstack2_thread.csv", "~/cumbia_frametrack2.csv")

In [3]:
def overhead_per_function(capture1, capture2):
  names1 = capture1.function_names()
  names2 = capture2.function_names()
  names = set(names1).intersection(set(names2))
  names.remove("???")
  capture1.filter_functions(names)
  capture2.filter_functions(names)

  avg_frame_time1 = capture1.avg_frame_time()
  avg_frame_time2 = capture2.avg_frame_time()

  time_per_frame1 = capture1.time_per_frame()
  time_per_frame2 = capture2.time_per_frame()

  result = []
  for name in names:
    time1 = time_per_frame1[name]
    time2 = time_per_frame2[name]
    result.append((name, (time1 - time2) / (avg_frame_time1 - avg_frame_time2)))
  return result


In [4]:
overheads = overhead_per_function(capture1, capture2)

In [5]:
overheads.sort(key = lambda x: -x[1])

In [6]:
def test(capture1, rates1, capture2, rates2, name):
  n1 = capture1.total_callstacks()
  n2 = capture2.total_callstacks()
  p1 = rates1[name] 
  p2 = rates2[name]  
  f1 = capture1.avg_frame_time()
  f2 = capture2.avg_frame_time()

  stat = f1*p1 - f2*p2
  stat_var = f1*p1*(f1-f1*p1)/n1 + f2*p2*(f2-f2*p2)/n2
  stat = stat / np.sqrt(stat_var)
  p = scipy.stats.norm().cdf(stat)
  return 1-p

  

In [7]:
names1 = capture1.function_names()
names2 = capture2.function_names()
rate1 = capture1.get_exclusive_rate()
rate2 = capture2.get_exclusive_rate()
names = set(names1).intersection(set(names2))
pvalues = [(name, test(capture1, rate1, capture2, rate2, name)) for name in names]

/tmp/ipykernel_2229794/3252247193.py:11: RuntimeWarning: invalid value encountered in double_scalars
  stat = stat / np.sqrt(stat_var)


In [8]:
reject, corrected,_,_ = multipletests([pvalue for name, pvalue in pvalues])

/usr/local/google/home/avanesov/.local/lib/python3.9/site-packages/statsmodels/stats/multitest.py:177: RuntimeWarning: divide by zero encountered in log1p
  np.log1p(-pvals))


In [9]:
np.array(reject)

array([False, False, False, ..., False, False, False])

In [10]:
namesnp = np.array([name for name, pvalue in pvalues])
rejected_names = namesnp[np.array(reject)]

In [11]:
[rate1[name] * capture1.total_callstacks() for name in rejected_names]

[176.0, 705.0]

In [12]:
rejected_names

array(['eva::concurrent::thread::current()',
       'eva::concurrent::dispatching::refresh_dependencies()'],
      dtype='<U10640')

In [13]:
overheads[0]

('inflate', 0.39241452134279214)